<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Compile data for Mason Water Budget**

By: Carli Beisel

Created: August 2024

**Variables:**

*qin* = streamflow at upstream gauge on reach

*qout* = streamflow at downstream gauge on reach

*sr* = surface runoff from agricultural irrigation

*gw* = groundwater

*evap* = potential open water evaporation

In [ ]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
## --------------- ##
##  LOAD DATASETS  ##
## --------------- ##

# Streamflow
drainflow = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/drainflow_out/final_monthly_flow.csv')
drainflow['Month'] = pd.to_datetime(drainflow['Month'], format='%Y-%m')
drainflow['Reach'] = drainflow['Reach'].replace('Noble Drain', 'Noble Reach')

# Precipitation
precip = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/precip_out/reach_precip_monthly.csv')
precip['Reach'] = precip['Reach'].replace({
    'Reach 1': 'Reach 0',
    'Reach 2': 'Reach 1',
    'Reach 3': 'Reach 2',
    'Reach 4': 'Reach 3'
})
# add Reach 1 & 2 together REPLACE LATER
precip.loc[precip['Reach'] == 'Reach 1', 'Total Monthly Precip (AF)'] = (
    precip[precip['Reach'] == 'Reach 0'].set_index('Month')['Total Monthly Precip (AF)'] +
    precip[precip['Reach'] == 'Reach 1'].set_index('Month')['Total Monthly Precip (AF)']
).values


# Evaporation (PET)
pet = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/pet_out/reach_pet.csv')
pet['Month'] = pd.to_datetime(pet['Month'].astype(str) + '-2024', format='%m-%Y', errors='coerce')
pet['Reach'] = pet['Reach'].replace({
    'Reach 1': 'Reach 0',
    'Reach 2': 'Reach 1',
    'Reach 3': 'Reach 2',
    'Reach 4': 'Reach 3'
})
# add Reach 1 & 2 together REPLACE LATER
pet.loc[pet['Reach'] == 'Reach 1', 'Reach Evap (AF)'] = (
    pet[pet['Reach'] == 'Reach 0'].set_index('Month')['Reach Evap (AF)'] +
    pet[pet['Reach'] == 'Reach 1'].set_index('Month')['Reach Evap (AF)']
).values



# Surface Runoff
surface_runoff = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/irrigation_runoff_out/reach_runoff_final.csv')
surface_runoff.rename(columns={'runoff(af)': 'SR (AF)'}, inplace=True)

# Correct month formatting for surface runoff
month_mapping = {
    "jan": "01", "feb": "02", "mar": "03", "apr": "04",
    "may": "05", "jun": "06", "jul": "07", "aug": "08",
    "sep": "09", "oct": "10", "nov": "11", "dec": "12"
}
surface_runoff['Month'] = surface_runoff['month'].str.lower().map(month_mapping)
surface_runoff['Month'] = pd.to_datetime("2024-" + surface_runoff['Month'], format='%Y-%m', errors='coerce')

In [58]:
surface_runoff

,Reach,month,runoff(af),Month
0,Noble Drain,apr,138.287698,2024-04-01
1,Noble Drain,aug,247.976204,2024-08-01
2,Noble Drain,dec,87.055424,2024-12-01
3,Noble Drain,feb,94.890627,2024-02-01
4,Noble Drain,jan,92.389697,2024-01-01
...,...,...,...,...
67,Solomon Reach,mar,101.139056,2024-03-01
68,Solomon Reach,may,214.336396,2024-05-01
69,Solomon Reach,nov,91.784895,2024-11-01
70,Solomon Reach,oct,123.394269,2024-10-01


In [60]:
## ------------------------------ ##
##   Merge into one .csv file (KAF)
## ------------------------------ ##

drainflow_selected = drainflow[['Reach', 'Month', 'Qin', 'Qout']]

pet_selected = pet[['Reach', 'Month', 'Reach Evap (AF)']]

precip_selected = precip[['Reach', 'Month', 'Total Monthly Precip (AF)']]
precip_selected['Month'] = pd.to_datetime(precip_selected['Month'], errors='coerce')

surface_runoff_selected = surface_runoff[['Reach', 'Month', 'SR (AF)']]
surface_runoff_selected['Reach'] = surface_runoff['Reach'].replace('Noble Drain', 'Noble Reach')

# Merge the DataFrames on "Reach" and "Month"
merged_df = drainflow_selected.merge(pet_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(precip_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(surface_runoff_selected, on=['Reach', 'Month'], how='outer')


# Fill Noble Reach ET + Precip with 0 for now
columns_to_replace = ["Reach Evap (AF)", "Total Monthly Precip (AF)"]
noble_reach_mask = merged_df['Reach'] == 'Noble Reach'

# Check if the columns exist in the DataFrame
for col in columns_to_replace:
    if col in merged_df.columns:
        merged_df.loc[noble_reach_mask, col] = merged_df.loc[noble_reach_mask, col].fillna(0)

# Display the final merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_af.csv')

<ipython-input-60-0df95032fb90>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_selected['Month'] = pd.to_datetime(precip_selected['Month'], errors='coerce')
<ipython-input-60-0df95032fb90>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surface_runoff_selected['Reach'] = surface_runoff['Reach'].replace('Noble Drain', 'Noble Reach')


In [61]:
# ---------------------------- #
# Calculate Water Budget
# --------------------------- #

# with Runoff

def calculate_wb_residual(df):
    """
    Calculate the water budget residual (Qout - Qin) in acre-feet (AF) for each Gauge at each Month.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Gauge', 'Month', 'Qin', and 'Qout' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Flow Difference (AF)' column.
    """
    # Ensure 'Qin' and 'Qout' are numeric
    df['Qin (AF)'] = pd.to_numeric(df['Qin'], errors='coerce')
    df['Qout (AF)'] = pd.to_numeric(df['Qout'], errors='coerce')
    df['Precip (AF)'] = pd.to_numeric(df['Total Monthly Precip (AF)'], errors='coerce')
    df['Evap (AF)'] = pd.to_numeric(df['Reach Evap (AF)'], errors='coerce')
    df['SR (AF)'] = pd.to_numeric(df['SR (AF)'], errors='coerce')

    # Calculate the flow difference ( Residual = output - input )
    df['Residual (AF)'] = (df['Qout (AF)'] + df['Evap (AF)']) - (df['Qin (AF)'] + df['Precip (AF)']+ df["SR (AF)"])

    return df

final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_af.csv')
residual = calculate_wb_residual(final_df)
residual = residual.drop_duplicates()
residual.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_af.csv', index = False)

In [55]:
# ---------------------------- #
# Calculate Water Budget
# --------------------------- #

# NO RUNOFF

def calculate_wb_residual(df):
    """
    Calculate the water budget residual (Qout - Qin) in acre-feet (AF) for each Gauge at each Month.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Gauge', 'Month', 'Qin', and 'Qout' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Flow Difference (AF)' column.
    """
    # Ensure 'Qin' and 'Qout' are numeric
    df['Qin (AF)'] = pd.to_numeric(df['Qin'], errors='coerce')
    df['Qout (AF)'] = pd.to_numeric(df['Qout'], errors='coerce')
    df['Precip (AF)'] = pd.to_numeric(df['Total Monthly Precip (AF)'], errors='coerce')
    df['Evap (AF)'] = pd.to_numeric(df['Reach Evap (AF)'], errors='coerce')

    # Calculate the flow difference ( Residual = output - input )
    df['Residual (AF)'] = (df['Qout (AF)'] + df['Evap (AF)']) - (df['Qin (AF)'] + df['Precip (AF)'])

    return df

final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_af.csv')
residual = calculate_wb_residual(final_df)
residual = residual.drop_duplicates()
residual.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_norunoff.csv', index = False)